<a href="https://colab.research.google.com/github/fevdaele/Machine_Learning_course_UGent_D012554_kaggle/blob/master/PredictionsFemke_12052020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt;
import numpy as np;
import pandas as pd;
import seaborn as sns;
sns.set_context("notebook", font_scale=1.4);
sns.set_style("whitegrid");
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

### **Load train and dataset**

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

### **Data pre processing**

In [0]:
data_train = trainset.copy()
label_column = data_train.pop('label')

data_test = testset.copy()
index_column = data_test.pop('index')

**Scaling of trainset**

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_train = scaler.fit_transform(data_train)


#plt.figure(figsize=(12,8))
#trainset_scaled.boxplot(vert=False)
#plt.show()

**Scaling of testset**

In [0]:
data_test = scaler.transform(data_test)
#plt.figure(figsize=(12,8))
#testset_scaled.boxplot(vert=False)
#plt.show()


### ***Model selection and validation***

In [58]:
from sklearn.svm import SVC

y = label_column
X = data_train

model = SVC(kernel='poly',C=1,degree=2)

score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)


0.7928429287328369


### ***Model optimization***

In [59]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn import metrics

X = data_train
y = label_column


search_space_C = [0.001, 0.01, 0.1, 1, 10]
search_space_degree = [1, 2, 3, 4, 5, 6]
param_grid = {'C': search_space_C, 'degree' : search_space_degree}
grid_search = GridSearchCV(model, param_grid, scoring = 'roc_auc', cv=10)
grid_search.fit(X, y)

print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)


SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
{'C': 10, 'degree': 3}
0.8867387841516281


In [60]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn import metrics

X = data_train
y = label_column


search_space_C = [10, 40, 60, 90, 100]
search_space_degree = [3, 4, 5, 6]
param_grid = {'C': search_space_C, 'degree' : search_space_degree}
grid_search = GridSearchCV(model, param_grid, scoring = 'roc_auc', cv=10)
grid_search.fit(X, y)

print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
{'C': 100, 'degree': 3}
0.9142188148243193


**Define and validate model with optimized hyper parameter**

In [64]:
model = SVC(kernel='poly',C=100,degree=3, probability=True)
score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)

0.9142188148243193


### ***Fit model on trainset***

In [65]:
model.fit(X,y)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

### ***Make predictions***

In [66]:
predictions = model.predict_proba(data_test)
print(predictions)

[[0.95787306 0.04212694]
 [0.61638014 0.38361986]
 [0.73710714 0.26289286]
 ...
 [0.21151822 0.78848178]
 [0.22508808 0.77491192]
 [0.99799438 0.00200562]]


**Create new dataframe to load predictions**

In [67]:
PredictionsFemke = pd.DataFrame(columns=['index','label'])
PredictionsFemke['index'] = index_column
PredictionsFemke['label'] = predictions[:,1]

print(PredictionsFemke)

       index     label
0          0  0.042127
1          1  0.383620
2          2  0.262893
3          3  0.821972
4          4  0.970660
...      ...       ...
12887  12887  0.322001
12888  12888  0.707277
12889  12889  0.788482
12890  12890  0.774912
12891  12891  0.002006

[12892 rows x 2 columns]


### **Save predictions in csv file**

In [0]:
filename = "PredictionsFemke12052020.csv"

#make sure to not write the Pandas index column (index=False)
PredictionsFemke.to_csv(filename,index=False)